# Неделя 1. Подготовка данных к анализу и построению моделей

In [1]:
%reload_ext watermark
%watermark -v -m -p numpy,scipy,pandas,matplotlib,statsmodels,sklearn -g

CPython 3.7.0
IPython 7.0.1

numpy 1.15.3
scipy 1.1.0
pandas 0.23.4
matplotlib 3.0.1
statsmodels 0.9.0
sklearn 0.20.0

compiler   : Clang 6.0 (clang-600.0.57)
system     : Darwin
release    : 18.0.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit
Git hash   : 6c444d24686d4f3351c0b8133d243f3f12a577bb


In [128]:
import csv
import pickle
import string
from collections import Counter, defaultdict
from functools import partial
from itertools import chain, count, zip_longest
from pathlib import Path
from typing import Any, Dict, List, Tuple

import numpy
import pandas
from more_itertools import chunked, sliced
from pandas import DataFrame
from scipy.sparse import csr_matrix

Посмотрим на один из файлов с данными о посещенных пользователем (номер 31) веб-страницах:

In [3]:
user_frame_31 = pandas.read_csv(Path('10users') / 'user0031.csv')

In [4]:
user_frame_31.head()

,timestamp,site
0,2013-11-15 08:12:07,fpdownload2.macromedia.com
1,2013-11-15 08:12:17,laposte.net
2,2013-11-15 08:12:17,www.laposte.net
3,2013-11-15 08:12:17,www.google.com
4,2013-11-15 08:12:18,www.laposte.net


Поставим задачу классификации: идентифицировать пользователя по сессии из 10 подряд посещенных сайтов. Объектом в этой задаче будет сессия из 10 сайтов, последовательно посещенных одним и тем же пользователем, признаками – индексы этих 10 сайтов (чуть позже здесь появится "мешок" сайтов, подход Bag of Words). Целевым классом будет id пользователя.

## Часть 1. Подготовка обучающей выборки

Реализуйте функцию `prepare_train_set`, которая принимает на вход путь к каталогу с CSV-файлами `path_to_csv_files` и параметр `session_length` – длину сессии, а возвращает 2 объекта:

* `DataFrame`, в котором строки соответствуют уникальным сессиям из `session_length` сайтов, `session_length` столбцов – индексам этих `session_length` сайтов и последний столбец – ID пользователя
* частотный словарь сайтов вида `{'site_string': [site_id, site_freq]}`, например для недавнего игрушечного примера это будет `{'vk.com': (1, 2), 'google.com': (2, 2), 'yandex.ru': (3, 3), 'facebook.com': (4, 1)}`

Детали:

* Смотрите чуть ниже пример вывода, что должна возвращать функция
* Используйте `glob` (или аналоги) для обхода файлов в каталоге. Для определенности, отсортируйте список файлов лексикографически. Удобно использовать `tqdm_notebook` (или просто `tqdm` в случае python-скрипта) для отслеживания числа выполненных итераций цикла
* Создайте частотный словарь уникальных сайтов (вида `{'site_string': (site_id, site_freq)}`) и заполняйте его по ходу чтения файлов. Начните с 1
* Рекомендуется меньшие индексы давать более часто попадающимся сайтам (приницип наименьшего описания)
* Не делайте entity recognition, считайте google.com, http://www.google.com и www.google.com разными сайтами (подключить entity recognition можно уже в рамках индивидуальной работы над проектом)
* Скорее всего в файле число записей не кратно числу `session_length`. Тогда последняя сессия будет короче. Остаток заполняйте нулями. То есть если в файле 24 записи и сессии длины 10, то 3 сессия будет состоять из 4 сайтов, и ей мы сопоставим вектор `[site1_id, site2_id, site3_id, site4_id, 0, 0, 0, 0, 0, 0, user_id]`
* В итоге некоторые сессии могут повторяться – оставьте как есть, не удаляйте дубликаты. Если в двух сессиях все сайты одинаковы, но сессии принадлежат разным пользователям, то тоже оставляйте как есть, это естественная неопределенность в данных
* Не оставляйте в частотном словаре сайт 0 (уже в конце, когда функция возвращает этот словарь)
* 150 файлов из `capstone_websites_data/150users/` у меня обработались за 1.7 секунды, но многое, конечно, зависит от реализации функции и от используемого железа. И вообще, первая реализация скорее всего будет не самой эффективной, дальше можно заняться профилированием (особенно если планируете запускать этот код для 3000 пользователей). Также эффективная реализация этой функции поможет нам на следующей неделе.

In [65]:
def prepare_train_set(path: Path, session_length: int = 10) -> Tuple[DataFrame, Dict[str, Tuple[int, int]]]:
    # List of visited sites per each user.
    user_sites: List[Tuple[int, List[str]]] = [
        (int(csv_path.stem[-4:]), [row['site'] for row in csv.DictReader(csv_path.open('rt'))])
        for csv_path in sorted(path.glob('*.csv'))
    ]
        
    # Count visits per each site.
    site_frequencies = Counter(chain.from_iterable(sites for _, sites in user_sites))
    
    # Assign an incremental ID to each unique site.
    site_ids: Dict[str, int] = dict(zip(site_frequencies, count(start=1)))
        
    frame = DataFrame([
        # User ID and each of `session_length` visited sites.
        {'user_id': user_id, **{
            # Assign `0` if session is too short.
            f'site_{i:02d}': site_ids.get(site, 0)
            for i, site in zip_longest(range(1, session_length + 1), session)
        }}
        for user_id, sites in user_sites
        for session in sliced(sites, session_length)  # split user's sites into sessions
    ], dtype=int)
    frequency_dict = {
        site: (site_id, site_frequencies[site]) 
        for site, site_id in site_ids.items()
    }
    return frame, frequency_dict

In [73]:
frame_3users, frequencies_3users = prepare_train_set(Path('3users'), session_length=10)

In [74]:
frame_3users

,site_01,site_02,site_03,site_04,site_05,site_06,site_07,site_08,site_09,site_10,user_id
0,1,2,2,3,2,4,5,6,7,8,1
1,1,4,4,4,0,0,0,0,0,0,1
2,1,2,9,9,2,0,0,0,0,0,2
3,10,4,2,4,2,4,4,6,11,10,3
4,10,4,2,0,0,0,0,0,0,0,3


In [72]:
frequencies_3users

{'vk.com': (1, 3),
 'oracle.com': (2, 8),
 'geo.mozilla.org': (3, 1),
 'google.com': (4, 9),
 'accounts.google.com': (5, 1),
 'mail.google.com': (6, 2),
 'apis.google.com': (7, 1),
 'plus.google.com': (8, 1),
 'football.kulichki.ru': (9, 2),
 'meduza.io': (10, 3),
 'yandex.ru': (11, 1)}

Примените полученную функцию к данным по 10 пользователям.

In [75]:
frame_10users, frequencies_10users = prepare_train_set(Path('10users'))

In [86]:
frame_10users.head()

,site_01,site_02,site_03,site_04,site_05,site_06,site_07,site_08,site_09,site_10,user_id
0,1,2,3,4,3,3,4,3,5,3,31
1,6,7,8,9,3,10,11,12,13,14,31
2,14,4,14,14,15,16,6,17,18,14,31
3,19,20,19,14,14,14,14,21,22,23,31
4,24,14,15,25,26,27,28,29,30,29,31


### Вопрос 1. Сколько уникальных сессий из 10 сайтов в выборке с 10 пользователями?

In [91]:
def write_answer(name: str, value: Any):
    with open(f'{name}.txt', 'wt') as fp:
        fp.write(str(value))
    return value

In [92]:
write_answer('answer1_1', len(frame_10users))

14061

### Вопрос 2. Сколько всего уникальных сайтов в выборке из 10 пользователей?

In [93]:
write_answer('answer1_2', len(frequencies_10users))

4913

Примените полученную функцию к данным по 150 пользователям.

In [89]:
frame_150users, frequencies_150users = prepare_train_set(Path('150users'))

### Вопрос 3. Сколько уникальных сессий из 10 сайтов в выборке с 150 пользователями?

In [94]:
write_answer('answer1_3', len(frame_150users))

137019

### Вопрос 4. Сколько всего уникальных сайтов в выборке из 150 пользователей?

In [95]:
write_answer('answer1_4', len(frequencies_150users))

27797

### Вопрос 5. Каковы топ-10 самых популярных сайтов среди посещенных 150 пользователями?

In [99]:
answer1_5_partial = sorted([
    (frequency, site)
    for site, (_, frequency) in frequencies_150users.items()
], reverse=True)[:10]

In [101]:
write_answer('answer1_5', ' '.join(site for _, site in answer1_5_partial))

'www.google.fr www.google.com www.facebook.com apis.google.com s.youtube.com clients1.google.com mail.google.com plus.google.com safebrowsing-cache.google.com www.youtube.com'

## Часть 2. Работа с разреженным форматом данных

In [132]:
def prepare_train_set_2(
    path: Path, 
    session_length: int = 10,
    field_names: List[str] = None,
    delimiter: str = ',',
) -> Tuple[csr_matrix, numpy.ndarray]:
    user_sessions: List[Tuple[int, Counter]] = [
        (int(csv_path.stem[-4:]), Counter(session))
        for csv_path in sorted(path.glob('*.csv'))
        for session in chunked((
            row['site']
            for row in csv.DictReader(csv_path.open('rt'), fieldnames=field_names, delimiter=delimiter)
        ), session_length)
    ]
    
    # Assign an incremental ID to each unique site.
    site_ids: Dict[str, int] = dict(zip(set(chain.from_iterable(session for _, session in user_sessions)), count()))
        
    x = csr_matrix((
        [value for _, session in user_sessions for value in session.values()],  # values are visit counts
        (
            [i for i, (_, session) in enumerate(user_sessions) for value in session.values()],  # row indices are session numbers
            [site_ids[site] for _, session in user_sessions for site in session],  # column indices are site IDs
        ),
    ))
    y = numpy.array([user_id for user_id, _ in user_sessions])
    
    return x, y

In [133]:
x_3users, y_3users = prepare_train_set_2(Path('3users'))
x_3users.todense()

matrix([[1, 1, 1, 0, 1, 1, 0, 3, 1, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0],
        [1, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 2, 4, 2, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]], dtype=int64)

In [126]:
%%time
x_10users, y_10users = prepare_train_set_2(Path('10users'))

CPU times: user 696 ms, sys: 14.2 ms, total: 710 ms
Wall time: 714 ms


In [129]:
with open('x_10users.pkl', 'wb') as fp:
    pickle.dump(x_10users, fp)
with open('y_10users.pkl', 'wb') as fp:
    pickle.dump(y_10users, fp)

In [127]:
%%time
x_150users, y_150users = prepare_train_set_2(Path('150users'))

CPU times: user 6.62 s, sys: 132 ms, total: 6.75 s
Wall time: 6.8 s


In [130]:
with open('x_150users.pkl', 'wb') as fp:
    pickle.dump(x_150users, fp)
with open('y_150users.pkl', 'wb') as fp:
    pickle.dump(y_150users, fp)

In [134]:
%%time
x_allcats, y_allcats = prepare_train_set_2(Path('allcats'), field_names=['user_id', 'ts', 'site'], delimiter=';')

CPU times: user 21.7 s, sys: 1.04 s, total: 22.7 s
Wall time: 23.3 s


In [137]:
with open('x_allcats.pkl', 'wb') as fp:
    pickle.dump(x_allcats, fp)
with open('y_allcats.pkl', 'wb') as fp:
    pickle.dump(y_allcats, fp)